In [ ]:
import os
import sys
import glob
import numpy as np
from PIL import Image
from tqdm import tqdm
from google.colab import drive

base_dir = "/content/van_gogh_gan"
!rm -rf {base_dir}
os.makedirs(base_dir, exist_ok=True)
os.makedirs(f"{base_dir}/data", exist_ok=True)
os.makedirs(f"{base_dir}/data/raw", exist_ok=True)
os.makedirs(f"{base_dir}/data/processed", exist_ok=True)
os.makedirs(f"{base_dir}/models", exist_ok=True)
os.makedirs(f"{base_dir}/results", exist_ok=True)
os.makedirs(f"{base_dir}/results/samples", exist_ok=True)

drive.mount('/content/drive', force_remount=True)
drive_dir = "/content/drive/MyDrive/van_gogh_gan"
os.makedirs(drive_dir, exist_ok=True)

print("Setting up StyleGAN3 environment...")

!pip install -q ninja scipy opencv-python matplotlib lpips kagglehub torch torchvision

!rm -rf stylegan3
!git clone https://github.com/NVlabs/stylegan3.git
sys.path.append(os.path.abspath("stylegan3"))

print("Downloading Van Gogh dataset...")
import kagglehub
dataset_path = kagglehub.dataset_download("ipythonx/van-gogh-paintings")

print("Processing images to 256x256...")
processed_dir = f"{base_dir}/data/processed"

paintings = []
for ext in ['.jpg', '.jpeg', '.png']:
    paintings.extend(glob.glob(f"{dataset_path}/**/*{ext}", recursive=True))

print(f"Found {len(paintings)} images to process")

for i, img_path in enumerate(tqdm(paintings)):
    try:
        img = Image.open(img_path).convert('RGB')

        width, height = img.size
        size = min(width, height)
        left = (width - size) // 2
        top = (height - size) // 2
        img = img.crop((left, top, left + size, top + size))

        img = img.resize((256, 256), Image.LANCZOS)

        img.save(f"{processed_dir}/img_{i:04d}.png")

        if i % 100 == 0 and i > 0:
            print(f"Processed {i}/{len(paintings)} images")
    except Exception as e:
        print(f"Error processing {img_path}: {e}")

print("Creating dataset zip...")
!python stylegan3/dataset_tool.py --source={processed_dir} --dest={base_dir}/data/vangogh.zip

print("Downloading compatible pre-trained model...")
!wget -nc https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-ffhqu-256x256.pkl \
     -O {base_dir}/models/stylegan3_pretrained.pkl

print("Starting fine-tuning with StyleGAN3...")
!python stylegan3/train.py --outdir={base_dir}/models \
    --data={base_dir}/data/vangogh.zip \
    --resume={base_dir}/models/stylegan3_pretrained.pkl \
    --gpus=1 --batch=16 --gamma=10 --mirror=1 \
    --snap=50 --kimg=1000 \
    --metrics=none \
    --cfg=stylegan3-r \
    --cbase=16384

model_snapshots = glob.glob(f"{base_dir}/models/**/network-snapshot-*.pkl", recursive=True)
if not model_snapshots:
    print("No model snapshots found. Checking for other model files...")
    model_snapshots = glob.glob(f"{base_dir}/models/**/*.pkl", recursive=True)
    model_snapshots = [m for m in model_snapshots if not m.endswith('stylegan3_pretrained.pkl')]

if model_snapshots:
    latest_model = max(model_snapshots, key=os.path.getctime)
    print(f"Using model: {latest_model}")

    print("Generating samples...")

    !python stylegan3/gen_images.py \
        --network={latest_model} \
        --seeds=0-15 \
        --outdir={base_dir}/results/samples \
        --trunc=0.7

    print("Backing up results to Google Drive...")
    !mkdir -p {drive_dir}/results/samples
    !cp {base_dir}/results/samples/*.png {drive_dir}/results/samples/ 2>/dev/null || true
    !cp {latest_model} {drive_dir}/

    print(f"\nAll results saved to Google Drive at: {drive_dir}")
    print(f"Generated images are in: {drive_dir}/results/samples/")
    print(f"The trained model is saved as: {drive_dir}/{os.path.basename(latest_model)}")
else:
    print("No trained model snapshots found. Generating samples with pretrained model...")

    !python stylegan3/gen_images.py \
        --network={base_dir}/models/stylegan3_pretrained.pkl \
        --seeds=0-15 \
        --outdir={base_dir}/results/samples \
        --trunc=0.7

    print("Backing up results to Google Drive...")
    !mkdir -p {drive_dir}/results/samples
    !cp {base_dir}/results/samples/*.png {drive_dir}/results/samples/ 2>/dev/null || true

    print(f"\nSamples generated with pretrained model saved to: {drive_dir}/results/samples/")

print("Pipeline completed!")

Mounted at /content/drive
Setting up StyleGAN3 environment...
Cloning into 'stylegan3'...
remote: Enumerating objects: 212, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 212 (delta 99), reused 90 (delta 90), pack-reused 49 (from 1)
Receiving objects: 100% (212/212), 4.16 MiB | 19.54 MiB/s, done.
Resolving deltas: 100% (108/108), done.
Processing images to 256x256...
Found 2025 images to process


  5%|▌         | 103/2025 [00:08<02:09, 14.84it/s]

Processed 100/2025 images


 10%|▉         | 201/2025 [00:16<02:28, 12.29it/s]

Processed 200/2025 images


...

Processed 1900/2025 images


 99%|█████████▉| 2002/2025 [01:52<00:02, 11.23it/s]

Processed 2000/2025 images


100%|██████████| 2025/2025 [01:53<00:00, 17.79it/s]


Creating dataset zip...
100% 2025/2025 [00:14<00:00, 136.28it/s]
--2025-05-14 11:30:42--  https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-ffhqu-256x256.pkl
Resolving api.ngc.nvidia.com (api.ngc.nvidia.com)... 52.10.4.28, 54.203.101.202
Connecting to api.ngc.nvidia.com (api.ngc.nvidia.com)|52.10.4.28|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://xfiles.ngc.nvidia.com/org/nvidia/team/research/models/stylegan3/versions/1/files/stylegan3-r-ffhqu-256x256.pkl?ssec-algo=AES256&versionId=IXmECuKSHP.HB4h6M9LD9rXen0blAI6D&ssec-key=HOzuokpWKp%2B9LrAL8AzTdgcZqBezpaXVW56aLp8goLxNbgTi8ukLAwiCy3KzXbgOvwwMazgvbuQ9tR5PrdnuJbrLUBRavJA36NpR1CKr3mkQpgMsciegJLA4ZVYRGWx4bGD7owKXobiWNUmpo%2Far3TOlsyJORVG75aZw3X1ncjZVk1kBfFYNGkRk2mguzm%2Bsuy9VK58mthW8YAbHP9kVGHYBVUj4b38R%2BtMI7r6XZrQuG7ZO9bZid8R8WMA6kEkrSVrV5WfFAGISd8mc7GiG26ivxdSa5BNniHHWoX3ldp2%2BkPcs8Txf6GwFN2swWCrXhpxS61tNFI6E1XF38hw6uCEt85SVAOcx9OV5Bo2pK7gMTEOGNd8QTs%2Bh

In [ ]:
!sed -i 's/betas=\[0,0.99\]/betas=[0.0,0.99]/g' stylegan3/train.py

print("Starting fine-tuning with StyleGAN3...")
!python stylegan3/train.py --outdir={base_dir}/models \
    --data={base_dir}/data/vangogh.zip \
    --resume={base_dir}/models/stylegan3_pretrained.pkl \
    --gpus=1 --batch=16 --gamma=10 --mirror=1 \
    --snap=50 --kimg=1000 --metrics=none \
    --cfg=stylegan3-r --cbase=16384

model_snapshots = glob.glob(f"{base_dir}/models/**/network-snapshot-*.pkl", recursive=True)
if not model_snapshots:
    print("No model snapshots found. Checking for other model files...")
    model_snapshots = glob.glob(f"{base_dir}/models/**/*.pkl", recursive=True)
    model_snapshots = [m for m in model_snapshots if not m.endswith('stylegan3_pretrained.pkl')]

if model_snapshots:
    latest_model = max(model_snapshots, key=os.path.getctime)
    print(f"Using model: {latest_model}")

    print("Generating samples...")

    !python stylegan3/gen_images.py \
        --network={latest_model} \
        --seeds=0-15 \
        --outdir={base_dir}/results/samples \
        --trunc=0.7

    print("Backing up results to Google Drive...")
    !mkdir -p {drive_dir}/results/samples
    !cp {base_dir}/results/samples/*.png {drive_dir}/results/samples/ 2>/dev/null || true
    !cp {latest_model} {drive_dir}/

    print(f"\nAll results saved to Google Drive at: {drive_dir}")
    print(f"Generated images are in: {drive_dir}/results/samples/")
    print(f"The trained model is saved as: {drive_dir}/{os.path.basename(latest_model)}")
else:
    print("No trained model snapshots found. Generating samples with pretrained model...")

    !python stylegan3/gen_images.py \
        --network={base_dir}/models/stylegan3_pretrained.pkl \
        --seeds=0-15 \
        --outdir={base_dir}/results/samples \
        --trunc=0.7

    print("Backing up results to Google Drive...")
    !mkdir -p {drive_dir}/results/samples
    !cp {base_dir}/results/samples/*.png {drive_dir}/results/samples/ 2>/dev/null || true

    print(f"\nSamples generated with pretrained model saved to: {drive_dir}/results/samples/")

print("Pipeline completed!")

Starting fine-tuning with StyleGAN3...

Training options:
{
  "G_kwargs": {
    "class_name": "training.networks_stylegan3.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "channel_base": 32768,
    "channel_max": 1024,
    "magnitude_ema_beta": 0.9994456359721023,
    "conv_kernel": 1,
    "use_radial_filters": true
  },
  "D_kwargs": {
    "class_name": "training.networks_stylegan2.Discriminator",
    "block_kwargs": {
      "freeze_layers": 0
    },
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 16384,
    "channel_max": 512
  },
  "G_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0.0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.0025
  },
  "D_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0.0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.002
  },
  "loss_kwargs": {
    "class_name": "training.loss.StyleGAN2Lo

# Метрики

In [2]:
# --- install & setup  -----------------------
!pip install -q ninja scipy opencv-python matplotlib lpips kagglehub torch torchvision

from google.colab import drive
drive.mount('/content/drive', force_remount=True)
DRIVE_DIR = "/content/drive/MyDrive/van_gogh_gan"
SNAPSHOT  = f"network-snapshot-001000.pkl"

import os, sys, glob, random, csv, torch, numpy as np, torchvision.transforms as T
from PIL import Image
import torch.nn.functional as F
from tqdm.auto import tqdm

# --- StyleGAN3 ---------------------------------------------------------------
!rm -rf stylegan3; git clone -q https://github.com/NVlabs/stylegan3.git
sys.path.append("/content/stylegan3")
import dnnlib, legacy

device = 'cuda' if torch.cuda.is_available() else 'cpu'
assert os.path.isfile(SNAPSHOT), "⛔️ snapshot not found!"

with dnnlib.util.open_url(SNAPSHOT) as f:
    G = legacy.load_network_pkl(f)['G_ema'].to(device).eval()

# --- prepare dataset ---------------------------------------------------------
import kagglehub
data_root = kagglehub.dataset_download("ipythonx/van-gogh-paintings")
real_pool = sum([glob.glob(f"{data_root}/**/*{ext}", recursive=True)
                 for ext in ('.jpg', '.jpeg', '.png')], [])

prep = T.Compose([T.ToTensor(), T.Normalize((0.5,)*3, (0.5,)*3)])

# --- metric helpers ----------------------------------------------------------
from lpips import LPIPS
lpips_fn = LPIPS(net='vgg').to(device).eval()

from torchvision.models import vgg16, VGG16_Weights
vgg = vgg16(weights=VGG16_Weights.IMAGENET1K_V1).features[:16].to(device).eval()
style_layers = {1, 6, 11}
def gram(x):
    b,c,h,w = x.shape
    f = x.view(b, c, h*w)
    return f @ f.transpose(1,2) / (c*h*w)
def style_loss(a, b):
    loss, g, r = 0.0, a, b
    with torch.no_grad():
        for i, layer in enumerate(vgg):
            g, r = layer(g), layer(r)
            if i in style_layers:
                loss += F.mse_loss(gram(g), gram(r))
    return loss.item()

# --- main loop ---------------------------------------------------------------
N_SAMPLES = 50
TRUNC     = 0.7

lpips_vals, style_vals = [], []
out_dir = "/content/samples"; os.makedirs(out_dir, exist_ok=True)

for seed in tqdm(range(N_SAMPLES), desc="Evaluating"):
    # generate
    z = torch.from_numpy(np.random.RandomState(seed).randn(1, G.z_dim)).to(device)
    gen = (G(z, None, truncation_psi=TRUNC, noise_mode='const').clamp(-1,1)+1)/2
    # pick real
    real_pil = Image.open(random.choice(real_pool)).convert('RGB')
    w,h = real_pil.size; s=min(w,h)
    real_pil = real_pil.crop(((w-s)//2, (h-s)//2, (w+s)//2, (h+s)//2)).resize((256,256), Image.LANCZOS)

    gen_t  = prep(T.ToPILImage()(gen[0].cpu())).unsqueeze(0).to(device)
    real_t = prep(real_pil).unsqueeze(0).to(device)

    lp = lpips_fn(gen_t, real_t).item()
    st = style_loss(gen_t, real_t)

    lpips_vals.append(lp); style_vals.append(st)

    # опционально сохраняем генерацию раз в 10, чтобы глазом проверить
    if seed % 10 == 0:
        T.ToPILImage()(gen[0].cpu()).save(f"{out_dir}/seed{seed:03d}.png")

# --- summary -----------------------------------------------------------------
import math, pandas as pd, json, pprint
mean_lp, std_lp = np.mean(lpips_vals), np.std(lpips_vals)
mean_st, std_st = np.mean(style_vals), np.std(style_vals)

print(f"\nLPIPS  : μ={mean_lp:.4f}  σ={std_lp:.4f}")
print(f"StyleL.: μ={mean_st:.4f}  σ={std_st:.4f}")

# сохраняем подробный лог
csv_path = f"{out_dir}/metrics_{N_SAMPLES}s.csv"
with open(csv_path, 'w', newline='') as f:
    w = csv.writer(f); w.writerow(['seed','lpips','style_loss'])
    w.writerows(zip(range(N_SAMPLES), lpips_vals, style_vals))
print(f"→ CSV с результатами: {csv_path}")

# бэкап в Drive
!cp -r /content/samples {DRIVE_DIR}/results/ 2>/dev/null || true
print(f"Everything backed up to {DRIVE_DIR}/results/")


Mounted at /content/drive


100%|██████████| 485M/485M [00:24<00:00, 21.1MB/s]

Extracting files...


Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 231MB/s]


...

Setting up PyTorch plugin "bias_act_plugin"... 

/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


Done.
Setting up PyTorch plugin "filtered_lrelu_plugin"... 

/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


Done.

LPIPS  : μ=0.7108  σ=0.0551
StyleL.: μ=0.0000  σ=0.0000
→ CSV с результатами: /content/samples/metrics_50s.csv
Everything backed up to /content/drive/MyDrive/van_gogh_gan/results/
